In [1]:
# Step 0. System setup
import sys
import glob
import os
import platform
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# if platform.system() == 'Windows':
#     server = '\\courageous'
# else:
#     server = '//Volumes'
# datapath = os.path.join('/Users', 'wenlianghou', 'Dropbox', 'BC Work', 'mortality', 'SSA2019')

In [2]:
 # Step 1. Load data
# cohort_file = os.path.join(datapath, "cohort.xlsx")

df_male = pd.read_excel("cohort.xlsx", index_col=0,sheet_name='male')
df_female = pd.read_excel("cohort.xlsx", index_col=0,sheet_name='female')
year_min = df_male.index.min()
year_max = df_male.index.max()
age_min = df_male.columns.min()
age_max = df_male.columns.max()

In [ ]:
# Step 2. Given interest and sex, calculate Annuity Factor
# interest = 5%
# male
r = 0.045
df_male_af5 = df_male
# AF calculation
df_mort = df_male
for birthyear in range(year_min,year_max+1):
    for age in range(age_min,age_max+1):
        # calc AF
        df_calc = df_mort.iloc[birthyear - year_min,age:].to_frame(name = 'q')
        df_calc['p'] = 1 - df_calc.q
        df_calc['cump'] = df_calc.p.cumprod(axis = 0)
        df_calc['discount'] = 1/ (1 + r)
        df_calc['discount'] = df_calc.discount.cumprod(axis = 0)
        df_calc['epv'] = df_calc.cump * df_calc.discount
        af = (df_calc.epv.sum())
        af = 1/af
        df_male_af5.iloc[birthyear - year_min,age] = af
%time

In [ ]:
df_male_af5.head()

In [ ]:
# with pd.ExcelWriter('annuity_male_' + str(r) + '_.xlsx') as writer:
#     df_male_af5.to_excel(writer, sheet_name=str(r))

# df_female_af5.to_csv('annuity_female_' + str(r) + '_.csv')

# with pd.ExcelWriter('annuity_female_total.xlsx') as writer:
#     for f in glob.glob(os.path.join('/Users/jacob.chuslo/BC/female', '*.csv')):
#         df = pd.read_csv(f)
#         df.to_excel(writer, sheet_name=os.path.basename(f))

In [ ]:
# with pd.ExcelWriter('annuity_male_' + str(r) + '_.xlsx') as writer:
#     df_male_af5.to_excel(writer, sheet_name=str(r))
    
# with pd.ExcelWriter(path, mode='a') as writer:
#     s.to_excel(writer, sheet_name='another sheet', index=False)

### overwrite excel sheet test

In [ ]:
dta = np.random.rand(1,3)
dta = dta.T
dta = pd.DataFrame(data=dta)

In [ ]:
from openpyxl import load_workbook

with pd.ExcelWriter('NEW.xlsx', engine='openpyxl', mode='a') as writer:
    dta.to_excel(writer, sheet_name='num3')

In [ ]:
dta = np.random.rand(1,4)
dta = dta.T
dta = pd.DataFrame(data=dta)

In [ ]:
#OVERWRITE WITHOUT DISRUPTING

book = load_workbook('NEW.xlsx')
writer = pd.ExcelWriter('NEW.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

# using a new dataframe...
dta.to_excel(writer, "num3") # other options: startrow = num, startcol = num
writer.save()

# Cumulative survival of male age 65 onwards at present day

### Optimal number of simulations at 95% and 90% confidence intervals

In [ ]:
import sys
import glob
import os
import platform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
df = pd.read_excel("cohort.xlsx", sheet_name="to_read", header=0, index_col=None)

In [ ]:
age = df['age'].to_numpy()
dead = df['dead at age'].to_numpy()
dead = dead.cumsum()

In [ ]:
plt.plot(dead,age)
# You can see that this is an "inverted CDF"

In [ ]:
# dead = dead.cumsum()

draws = np.random.uniform(0,1,1450)

values = np.interp(draws, dead, age)

values[:10]

values = np.ceil(values)

values[:10]

mean = values.mean()
var = values.var()

# Optimal number of simulation trials

#95% confidence
n1 = ( (1.96/0.05)*( var/mean ) )**2

#90% confidence
n2 = ( (1.645/0.05)*( var/mean ) )**2

print(len(values))
print(mean, var)
print(n1,n2)